# Besoin2

## implémentation random forest from scratch

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import json
import pickle
import matplotlib.pyplot as plt

data_prof = pd.read_csv('./Données/Data_Arbre.csv')
data_clean = pd.read_csv("./Données/data_clean.csv", encoding='utf-8', sep=";", decimal=",")

 # Sélectionner les colonnes pertinentes
selected_columns = ['haut_tot', 'haut_tronc', 'tronc_diam', 'fk_stadedev', 'fk_nomtech']
X = data_prof[selected_columns]
y = data_prof['age_estim']
X.shape

# Sélectionner les colonnes catégorielles à encoder
categorical_columns = ['fk_stadedev', 'fk_nomtech']

# Appliquer l'encodage sur les colonnes catégorielles
X_encoded = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
X_encoded.shape

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#  Créer et entraîner le modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)
X_train.shape

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

#  Afficher les coefficients du modèle
coefficients = pd.DataFrame(model.coef_, columns=['Coefficient'], index=X_encoded.columns)
print(coefficients)

Mean Squared Error: 5.237086588143302e+20
R^2 Score: -1.3730799368751908e+18
                       Coefficient
haut_tot                  0.227726
haut_tronc                2.285528
tronc_diam                0.167131
fk_stadedev_Jeune        -8.396638
fk_stadedev_senescent    29.282950
...                            ...
fk_nomtech_ULMJAP         4.392011
fk_nomtech_ULMMIN        12.667372
fk_nomtech_ULMRES        14.508682
fk_nomtech_ULMRESreb      0.237516
fk_nomtech_ULMRESsap      0.000000

[225 rows x 1 columns]


random forest

In [3]:
# Créer et entraîner le modèle de forêts aléatoires
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train_scaled, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_rf = model_rf.predict(X_test_scaled)

# Évaluer le modèle

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"Random Forest - Mean Squared Error: {mse_rf}")
print(f"Random Forest - R^2 Score: {r2_rf}")
print(f"Random Forest - RMSE: {rmse_rf}")

Random Forest - Mean Squared Error: 76.82860361273896
Random Forest - R^2 Score: 0.7985677486455527
Random Forest - RMSE: 8.765192731066383


CART

In [10]:
# Créer et entraîner le modèle CART
model_cart = DecisionTreeRegressor(max_depth=10, min_samples_split=2, random_state=42)
model_cart.fit(X_train_scaled, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_cart = model_cart.predict(X_test_scaled)

# Évaluer le modèle
mse_cart = mean_squared_error(y_test, y_pred_cart)
r2_cart = r2_score(y_test, y_pred_cart)
rmse_cart = np.sqrt(mse_cart)

print(f"CART - Mean Squared Error: {mse_cart}")
print(f"CART - R^2 Score: {r2_cart}")
print(f"CART - RMSE: {rmse_cart}")

CART - Mean Squared Error: 97.84297209297588
CART - R^2 Score: 0.743471451762393
CART - RMSE: 9.891560650017563


# tests


## test cart

In [24]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Définition de la fonction de variance pour la régression
def variance(y):
    return np.var(y)  # Calcule et retourne la variance de y

# Définition de la fonction de division des noeuds pour la régression
def split(X, y, feature_index, threshold):
    # Divise les indices en deux groupes : ceux où la caractéristique est <= seuil et ceux où elle est > seuil
    left_indices = np.where(X[:, feature_index] <= threshold)[0]
    right_indices = np.where(X[:, feature_index] > threshold)[0]
    return X[left_indices], X[right_indices], y[left_indices], y[right_indices]

# Définition de la classe Node pour l'arbre de régression
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index de la caractéristique utilisée pour la division
        self.threshold = threshold  # Seuil utilisé pour la division
        self.left = left  # Enfant gauche
        self.right = right  # Enfant droit
        self.value = value  # Valeur de la feuille si c'est une feuille

# Définition de la classe DecisionTree pour l'arbre de régression
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=None, criterion='variance'):
        self.root = None  # Racine de l'arbre
        self.min_samples_split = min_samples_split  # Nombre minimum d'échantillons pour diviser un nœud
        self.max_depth = max_depth  # Profondeur maximale de l'arbre
        self.criterion = criterion  # Critère utilisé pour la division (ici, la variance)

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)  # Construit l'arbre en appelant _grow_tree

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape  # Nombre d'échantillons et de caractéristiques
        # Critères pour créer une feuille : profondeur maximale atteinte, trop peu d'échantillons, ou variance nulle
        if (depth >= self.max_depth or n_samples < self.min_samples_split or len(set(y)) == 1):
            leaf_value = np.mean(y)  # Valeur de la feuille : moyenne des valeurs de y
            return Node(value=leaf_value)  # Retourne un nœud feuille

        best_feature, best_threshold = self._best_split(X, y, n_features)  # Trouve la meilleure division

        if best_feature is None:  # Si aucune division valide n'est trouvée
            return Node(value=np.mean(y))  # Retourne un nœud feuille

        # Divise les données selon la meilleure division trouvée
        left_X, right_X, left_y, right_y = split(X, y, best_feature, best_threshold)

        # Construit les sous-arbres récursivement
        left_child = self._grow_tree(left_X, left_y, depth + 1)
        right_child = self._grow_tree(right_X, right_y, depth + 1)

        # Retourne un nœud interne avec la meilleure division et les sous-arbres construits
        return Node(best_feature, best_threshold, left_child, right_child)

    def _best_split(self, X, y, n_features):
        best_var = np.inf  # Initialise la meilleure variance à l'infini
        split_idx, split_thresh = None, None  # Initialise les meilleurs index et seuil de division

        for feature_index in range(n_features):  # Parcourt toutes les caractéristiques
            thresholds = np.unique(X[:, feature_index])  # Obtient les seuils uniques pour cette caractéristique

            for threshold in thresholds:  # Parcourt tous les seuils possibles
                var = self._variance_index(X, y, feature_index, threshold)  # Calcule la variance pour cette division

                if var < best_var:  # Si cette division réduit la variance
                    best_var = var  # Met à jour la meilleure variance
                    split_idx = feature_index  # Met à jour le meilleur index de caractéristique
                    split_thresh = threshold  # Met à jour le meilleur seuil

        return split_idx, split_thresh  # Retourne le meilleur index de caractéristique et le meilleur seuil

    def _variance_index(self, X, y, feature_index, threshold):
        left_indices = np.where(X[:, feature_index] <= threshold)[0]  # Indices des échantillons dans le groupe de gauche
        right_indices = np.where(X[:, feature_index] > threshold)[0]  # Indices des échantillons dans le groupe de droite

        if len(left_indices) == 0 or len(right_indices) == 0:  # Si une division est vide, retourne une variance infinie
            return np.inf

        # Calcule les variances des groupes de gauche et de droite
        left_var = variance(y[left_indices])
        right_var = variance(y[right_indices])

        n_left, n_right = len(left_indices), len(right_indices)  # Nombre d'échantillons dans les groupes de gauche et de droite
        weighted_var = (n_left * left_var + n_right * right_var) / (n_left + n_right)  # Variance pondérée

        return weighted_var  # Retourne la variance pondérée

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])  # Prédit pour chaque échantillon dans X

    def _predict(self, inputs):
        node = self.root  # Commence à la racine
        while node.value is None:  # Tant que le nœud n'est pas une feuille
            if inputs[node.feature_index] <= node.threshold:  # Si la caractéristique est <= seuil
                node = node.left  # Va à gauche
            else:  # Sinon
                node = node.right  # Va à droite
        return node.value  # Retourne la valeur de la feuille

# Exemple de régression
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)  # Génère un jeu de données de régression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Divise les données en train/test

# Création du modèle DecisionTree pour la régression
model_regression = DecisionTree(min_samples_split=2, max_depth=10, criterion='variance')

# Entraînement du modèle sur les données d'entraînement
model_regression.fit(X_train, y_train)

# Prédictions sur les données de test
y_pred_regression = model_regression.predict(X_test)

# Évaluation du modèle
mse_regression = mean_squared_error(y_test, y_pred_regression)  # Calcule l'erreur quadratique moyenne
r2_regression = r2_score(y_test, y_pred_regression)  # Calcule le coefficient de détermination
rmse_regression = np.sqrt(mse_regression)  # Calcule la racine de l'erreur quadratique moyenne

# Affiche les résultats de l'évaluation
print(f"Regression - Mean Squared Error: {mse_regression}")
print(f"Regression - R^2 Score: {r2_regression}")
print(f"Regression - RMSE: {rmse_regression}")

Regression - Mean Squared Error: 16766.08356748525
Regression - R^2 Score: 0.5668651854282613
Regression - RMSE: 129.48391238870275


## tests random forest

In [26]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import mode

# Définition de la classe RandomForestClassifier
class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators  # Nombre d'arbres dans la forêt
        self.max_depth = max_depth  # Profondeur maximale des arbres
        self.min_samples_split = min_samples_split  # Nombre minimum d'échantillons requis pour diviser un nœud
        self.max_features = max_features  # Nombre de caractéristiques à considérer pour chaque division
        self.trees = []  # Liste pour stocker les arbres
        self.selected_features = []  # Liste pour stocker les indices des caractéristiques sélectionnées pour chaque arbre

    def fit(self, X, y):
        n_samples, n_features = X.shape  # Nombre d'échantillons et de caractéristiques
        if not self.max_features:
            self.max_features = int(np.sqrt(n_features))  # Définit max_features à la racine carrée du nombre de caractéristiques si non spécifié

        for _ in range(self.n_estimators):
            # Bootstrap sampling
            indices = np.random.choice(n_samples, n_samples, replace=True)  # Sélectionne des échantillons aléatoires avec remise
            X_bootstrap = X[indices]  # Échantillon bootstrap pour les caractéristiques
            y_bootstrap = y[indices]  # Échantillon bootstrap pour les étiquettes

            # Sélection aléatoire des caractéristiques
            selected_features = np.random.choice(n_features, self.max_features, replace=False)  # Sélectionne aléatoirement des caractéristiques
            self.selected_features.append(selected_features)  # Sauvegarde les indices des caractéristiques sélectionnées

            # Création et entraînement de l'arbre de décision
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split)  # Crée un arbre de décision
            tree.fit(X_bootstrap[:, selected_features], y_bootstrap)  # Entraîne l'arbre sur l'échantillon bootstrap et les caractéristiques sélectionnées

            self.trees.append(tree)  # Ajoute l'arbre à la forêt

    def predict(self, X):
        # Agréger les prédictions de chaque arbre
        predictions = np.zeros((X.shape[0], len(self.trees)))  # Crée un tableau pour stocker les prédictions de chaque arbre pour chaque échantillon
        for i, tree in enumerate(self.trees):
            selected_features = self.selected_features[i]  # Récupère les caractéristiques sélectionnées pour cet arbre
            predictions[:, i] = tree.predict(X[:, selected_features])  # Prédictions de l'arbre en utilisant les caractéristiques sélectionnées
        
        # Vote majoritaire
        final_predictions = mode(predictions, axis=1)[0].flatten()  # Utilise le vote majoritaire pour obtenir les prédictions finales
        return final_predictions.astype(int)  # Retourne les prédictions finales en tant qu'entiers

# Exemple d'utilisation
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)  # Génère un ensemble de données de classification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Divise les données en ensembles d'entraînement et de test

# Création du modèle de Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, max_features=None)  # Crée une instance de RandomForestClassifier

# Entraînement du modèle sur les données d'entraînement
rf_classifier.fit(X_train, y_train)

# Prédictions sur les données de test
y_pred = rf_classifier.predict(X_test)

# Évaluation du modèle
mse_rf = mean_squared_error(y_test, y_pred)  # Calcul de l'erreur quadratique moyenne
r2_rf = r2_score(y_test, y_pred)  # Calcul du coefficient de détermination
rmse_rf = np.sqrt(mse_rf)  # Calcul de la racine de l'erreur quadratique moyenne

print(f"Random Forest - Mean Squared Error: {mse_rf}")
print(f"Random Forest - R^2 Score: {r2_rf}")
print(f"Random Forest - RMSE: {rmse_rf}")


Random Forest - Mean Squared Error: 0.145
Random Forest - R^2 Score: 0.41714400562757503
Random Forest - RMSE: 0.3807886552931954


C:\Users\elean\AppData\Local\Temp\ipykernel_32068\2322386594.py:47: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  final_predictions = mode(predictions, axis=1)[0].flatten()  # Utilise le vote majoritaire pour obtenir les prédictions finales
